In [126]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import Transformer

In [22]:
def reproject_image(input_image, output_image, dst_crs='EPSG:4326'):
    with rasterio.open(input_image) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        with rasterio.open(output_image, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)


In [23]:
reproject_image('landsat2014_B4.TIF', 'landsat2014_B4_rp.TIF')
reproject_image('landsat2024_B4.TIF', 'landsat2024_B4_rp.TIF')
reproject_image('landsat2014_B5.TIF', 'landsat2014_B5_rp.TIF')
reproject_image('landsat2024_B5.TIF', 'landsat2024_B5_rp.TIF')


In [122]:
with rasterio.open('landsat2014_B5.TIF') as src:
    nir2014 = src.read().astype('float64')
    profile = src.profile
    transform = src.transform
    crs = src.crs
with rasterio.open('landsat2014_B4.TIF') as src:
    red2014 = src.read().astype('float64')
with rasterio.open('landsat2024_B5.TIF') as src:
    nir2024 = src.read().astype('float64')
with rasterio.open('landsat2024_B4.TIF') as src:
    red2024 = src.read().astype('float64')

In [53]:
def calculate_ndvi(nir, red):
    ndvi = (nir - red) / (nir + red)
    return ndvi

In [ ]:
ndvi2014 = calculate_ndvi(nir2014, red2014)
ndvi2024 = calculate_ndvi(nir2024, red2024)


In [55]:
nvdi_change = ndvi2024 - ndvi2014

In [56]:
classified_nvdi_change = np.where(nvdi_change>0.1,1,np.where(nvdi_change <-0.1,-1, 0))

In [60]:
classified_nvdi_change = classified_nvdi_change.squeeze()

In [ ]:
plt.imshow(classified_nvdi_change, cmap='RdYlGn')
plt.colorbar()
plt.title('Change in Land Use (2000-2023)')
plt.show()


In [109]:
with rasterio.open('nvdi_change.tif', 'w', **profile) as dst:
    dst.write(classified_nvdi_change.astype(rasterio.int32), 1)


In [145]:
def getRasterValue(xy, raster, transform):
    xy = list(xy)
    transformer = Transformer.from_crs('epsg:4326', crs, always_xy=True)
    long,lat = transformer.transform(xy[1], xy[0])
    row, col = ~transform * (long, lat)
    row, col = int(row), int(col)
    value = raster[row,col]
    return value


In [ ]:
coords = 36.5011886054449, 52.849342734743566
getRasterValue(coords, classified_nvdi_change, transform)